In [10]:
import json
from datetime import datetime , timedelta

import numpy as np
#from mocpy import MOC , STMOC
#from cdshealpix import healpix_to_skycoord
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.table import QTable
from astropy.samp import SAMPIntegratedClient
from astroplan import Observer
from matplotlib import pyplot as pl
from scipy.stats import norm

In [11]:
def samp_script_transmit(script):
    """ Sending an Aladin script over SAMP.

        Parameters
        ----------
        script : str
        Aladin script command
        
        Returns
        -------
        Transmit a Aladin command when SAMP Hub is running.
    """
    
    client = SAMPIntegratedClient()
    client.connect()
    params = {}

    message = {}
    message ["samp.mtype"] = "script.aladin.send"
    message ["samp.params"] = {'script': script}
    
    client.notify_all(message)
    client.disconnect()

In [12]:
url = "https://gracedb.ligo.org/api/superevents/S200316bj/files/LALInference.fits.gz"

In [13]:
script = 'load ' + url

In [14]:
from tkinter import messagebox

try:
    samp_script_transmit(script)
except:
    messagebox.showwarning("Title", "Before running a function, you have to define it through the 'def' command.")
    messagebox.showerror("Title", "Please, go back and run the code which defines 'samp_script_transmit'.") 

In [15]:
from matplotlib import pyplot as pl
from scipy.stats import norm

def dist_dist_along_line_of_sight(url, ra, dec):
    """Conditional distance distribution along a given 
    line of sight, which is the probability per unit
    distance under the assumption that the source is in a given direction."""
    
    filename = download_file(url, cache=True)
    
    
    h = hp.read_map(filename, h = True)
    h = dict(h[1])
    mean, std = h['DISTMEAN'], h['DISTSTD']

    r = np.linspace(mean-5*std, mean+5*std)
    m_dr = norm(mean, std).pdf(r)

    prob, distmu, distsigma, distnorm = hp.read_map(filename, 
                                                    field=range(4))
    npix = len(prob)
    nside = hp.npix2nside(npix)

    theta = 0.5 * np.pi - np.deg2rad(dec)
    phi = np.deg2rad(ra)
    
    ipix = hp.ang2pix(nside, theta, phi)
    
    dp_dr = r**2 * distnorm[ipix] * norm(
    distmu[ipix], distsigma[ipix]).pdf(r)
    
    return r, dp_dr, m_dr

In [16]:
from astropy.samp import SAMPIntegratedClient
client = SAMPIntegratedClient()

client.connect()

In [17]:
from tkinter import messagebox
from astropy.samp import SAMPIntegratedClient
client = SAMPIntegratedClient()

try:
     client.connect()
except:
    messagebox.showerror("Title", "Please, run Aladin.")

In [ ]:
class Receiver(object):
    def __init__(self, client):
        self.client = client
        self.received = False
    def receive_call(self, private_key, sender_id, msg_id, mtype, params, extra):
        self.params = params
        self.received = True
        self.client.reply(msg_id, {"samp.status": "samp.ok", "samp.result": {}})
    def receive_notification(self, private_key, sender_id, mtype, params, extra):
        self.params = params
        self.received = True

In [18]:
import time
%time
import tkinter
from pandas import DataFrame
from matplotlib.backends.backend_tkagg import (
    FigureCanvasTkAgg, NavigationToolbar2Tk)
# Implement the default Matplotlib key bindings.
from matplotlib.backend_bases import key_press_handler
from matplotlib.figure import Figure

import numpy as np

def embed_matlib(r, dp_dr, m_dr):
    
    
    data1 = {'distance (Mpc)': r,
         'conditional distance probability Mpc$ˆ{-1}$': dp_dr}
    df1 = DataFrame(data1,columns=['distance (Mpc)','conditional distance probability Mpc$ˆ{-1}$'])

    data2 =  {'distance (Mpc)': r,
         'marginal distance probability Mpc$ˆ{-1}$': m_dr}
    df2 = DataFrame(data2,columns=['distance (Mpc)','marginal distance probability Mpc$ˆ{-1}$'])

    
    root = tkinter.Tk()
    root.wm_title("Embedding in Tk")

    fig = Figure(figsize=(5, 4), dpi=100)
    t = np.arange(0, 3, .01)
    ax1 = fig.add_subplot(111)
    df1 = df1[['distance (Mpc)','conditional distance probability Mpc$ˆ{-1}$']].groupby('distance (Mpc)').sum()
    df1.plot(kind='line', legend=True, ax=ax1, color='b',fontsize=10)
    ax1.set_title('Probability analysis of the chosen pixel')
    df2 = df2[['distance (Mpc)','marginal distance probability Mpc$ˆ{-1}$']].groupby('distance (Mpc)').sum()
    df2.plot(kind='line', legend=True, ax=ax1, color='orange',fontsize=10)
    

    canvas = FigureCanvasTkAgg(fig, master=root)  # A tk.DrawingArea.
    canvas.draw()
    canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)

    toolbar = NavigationToolbar2Tk(canvas, root)
    toolbar.update()
    canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)


    def on_key_press(event):
        print("you pressed {}".format(event.key))
        key_press_handler(event, canvas, toolbar)


    canvas.mpl_connect("key_press_event", on_key_press)
    

    def _quit():
        root.quit()     # stops mainloop
        root.destroy()  # this is necessary on Windows to prevent
                    # Fatal Python Error: PyEval_RestoreThread: NULL tstate

    #label = tkinter.Label(text='Your current Sky Map is: ' + url, height=4)
    #label.pack(side=tkinter.TOP)
    
    labelText = tkinter.StringVar()
    labelText.set('Your current Sky Map is available here:\n' + url)
    labelDir = tkinter.Label(textvariable = labelText, height=5)
    labelDir.pack(side=tkinter.TOP)
    
    button = tkinter.Button(master=root, text="Quit", command=_quit)
    button.pack(side=tkinter.BOTTOM)

    tkinter.mainloop()
# If you put root.destroy() here, it will cause an error if the window is
# closed with the window manager.

CPU times: user 3 µs, sys: 3 µs, total: 6 µs
Wall time: 8.82 µs


In [21]:
import time
%time
import healpy as hp
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import norm
from tkinter import messagebox
from astropy.utils.data import download_file

try:
    position = Receiver(client)

    client.bind_receive_call(
    "coord.pointAt.sky", position.receive_call)
    client.bind_receive_notification(
    "coord.pointAt.sky", position.receive_notification)


    while True:
        if position.received == True:
            ra = float(position.params['ra'])
            dec = float(position.params['dec'])
        
            r, dp_dr, m_dr = dist_dist_along_line_of_sight(url, ra, dec)
        
            embed_matlib(r, dp_dr, m_dr)
            position.received = False    

except:
    messagebox.showwarning("Title", "Before running a function, you have to define it through the 'def' command.")
    messagebox.showerror("Title", "Please, go back and run the code which defines the 'Receiver' class.")        

CPU times: user 1e+03 ns, sys: 1e+03 ns, total: 2 µs
Wall time: 5.01 µs


In [ ]:
import tkinter
from tkinter import messagebox
# hide main window
root = tkinter.Tk()
messagebox.showwarning("Warning", "Click a position on the sky-map if you want more info!")

In [ ]:
from tkinter import messagebox

try:
    embed_matlib(r, dp_dr, m_dr)
except:
     #print("Please, run Aladin")
    messagebox.showwarning("Title", "Before running a function, you have to define it through the 'def' command.")
    messagebox.showerror("Title", "Please, go back and run the code which defines 'embed_matlib'.")